# André Lucas Fabbris de Toledo - RA: 1902777
# Lucas de Alencar Silva - RA: 1902989

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.regression import LinearRegression

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
spark = SparkSession.builder\
.master("local")\
.appName("Colab")\
.config('spark.ui.port', '4050')\
.getOrCreate()

In [ ]:
from pyspark.ml.feature import StringIndexer
import pyspark.sql.functions as F

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/predictive_maintenance.csv', inferSchema=True, header=True)

In [ ]:
df.show(10)

+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+------+------------+
|UDI|Product ID|Type|Air temperature [K]|Process temperature [K]|Rotational speed [rpm]|Torque [Nm]|Tool wear [min]|Target|Failure Type|
+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+------+------------+
|  1|    M14860|   M|              298.1|                  308.6|                  1551|       42.8|              0|     0|  No Failure|
|  2|    L47181|   L|              298.2|                  308.7|                  1408|       46.3|              3|     0|  No Failure|
|  3|    L47182|   L|              298.1|                  308.5|                  1498|       49.4|              5|     0|  No Failure|
|  4|    L47183|   L|              298.2|                  308.6|                  1433|       39.5|              7|     0|  No Failure|
|  5|    L47184|   L|              298.2|

1. Altere as variáveis de entrada inserindo Air temperature [K] e Tool wear [min] e mostre o erro R2 tendo como saída a variável temperatura do processo.

In [ ]:
indexer = StringIndexer(inputCol = "Failure Type", outputCol = 'Failure Type_cat')
indexed = indexer.fit(df).transform(df)

In [ ]:
vectorAssembler1 = VectorAssembler(inputCols=["Air temperature [K]", "Tool wear [min]"], outputCol="features")

In [ ]:
df_transformed1 = vectorAssembler1.transform(indexed)

In [ ]:
df_transformed1.select('features', 'Tool wear [min]', 'Tool wear [min]').show()

+------------+---------------+---------------+
|    features|Tool wear [min]|Tool wear [min]|
+------------+---------------+---------------+
| [298.1,0.0]|              0|              0|
| [298.2,3.0]|              3|              3|
| [298.1,5.0]|              5|              5|
| [298.2,7.0]|              7|              7|
| [298.2,9.0]|              9|              9|
|[298.1,11.0]|             11|             11|
|[298.1,14.0]|             14|             14|
|[298.1,16.0]|             16|             16|
|[298.3,18.0]|             18|             18|
|[298.5,21.0]|             21|             21|
|[298.4,24.0]|             24|             24|
|[298.6,29.0]|             29|             29|
|[298.6,34.0]|             34|             34|
|[298.6,37.0]|             37|             37|
|[298.6,40.0]|             40|             40|
|[298.6,42.0]|             42|             42|
|[298.6,44.0]|             44|             44|
|[298.7,47.0]|             47|             47|
|[298.8,50.0]

In [ ]:
treino, test = df_transformed1.randomSplit([0.7, 0.3])

In [ ]:
lr1 = LinearRegression(featuresCol='features', labelCol = 'Process temperature [K]')

In [ ]:
treino_modelo1 = lr1.fit(treino)

In [ ]:
resultado1 = treino_modelo1.evaluate(treino)
resultadoTest1 = treino_modelo1.evaluate(test)

In [ ]:
print('Erro R2:', resultado1.r2)
print('Erro R2:', resultadoTest1.r2)

Erro R2: 0.7658616502810621
Erro R2: 0.7714724772218031


2. Altere as variáveis de entrada inserindo Air temperature [K], Tool wear [min] e torque e mostre o erro R2 tendo como saída a variável temperatura do processo.

In [ ]:
vectorAssembler2 = VectorAssembler(inputCols=["Air temperature [K]", "Tool wear [min]", "Torque [Nm]"], outputCol="features")

In [ ]:
df_transformed2 = vectorAssembler2.transform(indexed)

In [ ]:
df_transformed2.select('features', 'Tool wear [min]', 'Tool wear [min]', 'Torque [Nm]').show()

+-----------------+---------------+---------------+-----------+
|         features|Tool wear [min]|Tool wear [min]|Torque [Nm]|
+-----------------+---------------+---------------+-----------+
| [298.1,0.0,42.8]|              0|              0|       42.8|
| [298.2,3.0,46.3]|              3|              3|       46.3|
| [298.1,5.0,49.4]|              5|              5|       49.4|
| [298.2,7.0,39.5]|              7|              7|       39.5|
| [298.2,9.0,40.0]|              9|              9|       40.0|
|[298.1,11.0,41.9]|             11|             11|       41.9|
|[298.1,14.0,42.4]|             14|             14|       42.4|
|[298.1,16.0,40.2]|             16|             16|       40.2|
|[298.3,18.0,28.6]|             18|             18|       28.6|
|[298.5,21.0,28.0]|             21|             21|       28.0|
|[298.4,24.0,23.9]|             24|             24|       23.9|
|[298.6,29.0,44.3]|             29|             29|       44.3|
|[298.6,34.0,51.1]|             34|     

In [ ]:
treino, test = df_transformed2.randomSplit([0.7, 0.3])

In [ ]:
lr2 = LinearRegression(featuresCol='features', labelCol = 'Process temperature [K]')

In [ ]:
treino_modelo2 = lr2.fit(treino)

In [ ]:
resultado2 = treino_modelo2.evaluate(treino)
resultadoTest2 = treino_modelo2.evaluate(test)

In [ ]:
print('Erro R2:', resultado2.r2)
print('Erro R2:', resultadoTest2.r2)

Erro R2: 0.7673043857746062
Erro R2: 0.7676595600965126


3. Altere as variáveis de entrada inserindo Air temperature [K], Tool wear [min], velocidade de rotação e torque e mostre o erro R2 tendo como saída a variável temperatura do processo.

In [ ]:
vectorAssembler3 = VectorAssembler(inputCols=["Air temperature [K]", "Tool wear [min]", "Rotational speed [rpm]", "Torque [Nm]"], outputCol="features")

In [ ]:
df_transformed3 = vectorAssembler3.transform(indexed)

In [ ]:
df_transformed3.select('features', 'Tool wear [min]', 'Tool wear [min]', 'Rotational speed [rpm]', 'Torque [Nm]').show()

+--------------------+---------------+---------------+----------------------+-----------+
|            features|Tool wear [min]|Tool wear [min]|Rotational speed [rpm]|Torque [Nm]|
+--------------------+---------------+---------------+----------------------+-----------+
|[298.1,0.0,1551.0...|              0|              0|                  1551|       42.8|
|[298.2,3.0,1408.0...|              3|              3|                  1408|       46.3|
|[298.1,5.0,1498.0...|              5|              5|                  1498|       49.4|
|[298.2,7.0,1433.0...|              7|              7|                  1433|       39.5|
|[298.2,9.0,1408.0...|              9|              9|                  1408|       40.0|
|[298.1,11.0,1425....|             11|             11|                  1425|       41.9|
|[298.1,14.0,1558....|             14|             14|                  1558|       42.4|
|[298.1,16.0,1527....|             16|             16|                  1527|       40.2|
|[298.3,18

In [ ]:
treino, test = df_transformed3.randomSplit([0.7, 0.3])

In [ ]:
lr3 = LinearRegression(featuresCol='features', labelCol = 'Process temperature [K]')

In [ ]:
treino_modelo3 = lr3.fit(treino)

In [ ]:
resultado3 = treino_modelo3.evaluate(treino)
resultadoTest3 = treino_modelo3.evaluate(test)

In [ ]:
print('Erro R2:', resultado3.r2)
print('Erro R2:', resultadoTest3.r2)

Erro R2: 0.7710328869841531
Erro R2: 0.7589450338607086


4. Alterando a variável de saída para desgaste da ferramenta e inserindo as variáveis de entrada Air temperature [K], Tool wear [min], velocidade de rotação e torque e alvo , é possível obter um r2 maior que 60% para os dados de treino e teste? Faça o experimento e análise.

In [ ]:
vectorAssembler4 = VectorAssembler(inputCols=["Air temperature [K]", "Tool wear [min]", "Rotational speed [rpm]", "Torque [Nm]", "Target"], outputCol="features")

In [ ]:
df_transformed4 = vectorAssembler4.transform(indexed)

In [ ]:
treino, test = df_transformed4.randomSplit([0.7, 0.3])

In [ ]:
lr4 = LinearRegression(featuresCol='features', labelCol = 'Tool wear [min]')

In [ ]:
treino_modelo4 = lr4.fit(treino)

In [ ]:
resultado4 = treino_modelo4.evaluate(treino)
resultadoTest4 = treino_modelo4.evaluate(test)

In [ ]:
print('Erro R2:', resultado4.r2)
print('Erro R2:', resultadoTest4.r2)

Erro R2: 1.0
Erro R2: 1.0


Resposta: É possivel sim ter um entendimento maior que 60%, visto que o R2 teve um retorno de 100%.